# Configuration Notes

Configuring device consists of four stages as shown in figure below, plus an initial set of reset instructions.

![hello](../doc/diagram.png)

# Config Notes

Variable $x$ refers to PLLs A or B.

Variable $y$ refers to CLKs 0 through 7.

## Input Stage

**Registers**: XTAL_CL, CLKIN_DIV
    
Input frequency: $f_{IN}$ determined by input source selected from one of CLKIN, XTAL, or VC.



## Synthesis Stage 1 (PLL)

**Registers**:
- $M_{PLL}$: MSNx_P1 (18 bits), MSNx_P2 (20 bits), MSNx_P3 (20 bits)
- FBx_INT


Generate PLL intermediate frequency.  Many documents refer to this signal as $f_{VCO}$, but I'm calling it $f_{PLLx}$.  Phase control is performed later in units of one fourth the period of this signal.  If I want high phase resolution, I should force this intermediate frequency as high as possible.
 
$f_{PLL} = f_{XTAL} \times M_{PLL}$

where

$M_{PLL} = a + \frac{b}{c}$ and must be in the range 15 - 90.

**How**: Given $f_{XTAL}$ and desired PLL frequency $f_{PLLx}$, start things off by computing $M_{PLL} \approx \frac{f_{PLLx}}{f_{XTAL}}$.  See below for a, b, and c parameters for encoding this number.

Valid $M_{PLL}$ ratios are 4, 6, 8, and any fractional value between (15 + 0/1,048,575) and (90 + 0/1,048,575).

If $M_{PLL}$ is an even integer then also set FBx_INT to 1 for reduced phase noise.

## Synthesis Stage 2 (MS)

**Registers**:
- MS?

Generate MultiSynth clock frequencies in the range 500 kHz and 150 MHz.

$f_{MSy} = f_{XTAL} \times \frac{M_{PLL}}{M_{MS}}$

where

$M_{MS} = p + \frac{q}{r}$ and must be in the range 6 - 1800.

Notes:
- MS6 and MS7 do not have parameters $q$ and $r$.
- CLK6 and CLK7 MultiSynth divide ratios $M_{MS,6}$ and $M_{MS,7}$ are restricted to even integer in the range 6 to 254.


Registers: MSx_P1 (18 bits), MSx_P2 (20 bits), MSx_P3 (20 bits)



## Output Stage

Final divider $R_y$ is a power of 2: 1, 2, 4, 8, ..., 128.

$f_{CLKy} = f_{MS,y}\times \frac{1}{R_y} = f_{XTAL} \times \frac{M_{PLL}}{M_{MS} \times R_{y}}$

For now I'm going to ignore $R_y$ and assume it's set to unity.

$f_{CLKy} = f_{XTAL} \times \frac{M_{PLL}}{M_{MS} }$

Both $M_{PLL}$ and $M_{MS}$ and are computed from an expression of the form $a + \frac{b}{c}$, which allows for a ratio of floats for great flexibility in choosing an output frequency, at the expense of potential jitter or phase noise.

If I wish to minimize phase noise then I should attempt to set $b=0$ and $c=1$, leaving only $a$ as a variable.  This ultimately means $M_{PLL}$ and $M_{MS}$ are each integers and I'm left with computing their ratio in order to set my output frequency.

# Multiplier Encoding

Frequency multiplier/dividers $M_{PLL}$ and $M_{MS}$ are each encoded using the following scheme:

$P_1 = 128 \times a + Floor\left(128 \times \frac{b}{c}\right) - 512$

$P_2 = 128 \times b - c \times Floor\left(128 \times \frac{b}{c}\right)$

$P_3 = c$


# Thoughts

Options:

- set $M_{PLL}$ to an integer (15 - 90) and $M_{MS}$ to an integer (6 - 1800)
- what if integers don't work for a scenario?  Am I better off selecting PLL or MS as fractional?
    - Fractional ratios introduce jitter (phase noise)
    - If I set PLL as fraction, then all downstream MS inherit that noise, but at least that noise is *common* across all downstream clocks.  That's a good thing right?
    
Maybe I should forget about fiddling with different frequencies, and instead focus on manipulating phase lags of individual clocks.

What is the minimum frequency resolution I want?  Better than 1 ppm for sure, relative to an output signal at 28.8 MHz.  That corresponds to 2.88 Hz resolution.  What if I shoot for 1 Hz?

What numbers do I get if I stick with even integers top and bottom?

### Rational Numbers and Fractions

Given that we're representing arbitrary floating-point numbers in the form $a + \frac{b}{c}$ I believe we'll see better performance on average if $a$ is always the `int` of the floating poijnt number in question.  The ratio of $b$ and $c$ is then used to represent a fractional value between 0 and 1.

### Worlflow

Start assuming that $M_{PLL}$ and $M_{MS}$ must both be integers.  Estimate those integers using Python `fractions` module.

If resulting output frequency is not close enough, estimate a floating point number for $M_{MS}$ that gets us closer.

If resulting frequency is still not good enough, estimate new floating point number for $M_{PLL}$.

The end. 

---

# Setup

In [1]:
import numpy as np

from snail.Si5351_Clock import clock

# Plan

In [2]:
f_XTAL = 25e6
f_CLK = 28.8e6

f_PLL_set = 800e6

In [3]:
# PLL as integer multiplier, MS as floating-point divider
M_PLL = int(f_PLL_set / f_XTAL)

M_MS = f_XTAL * M_PLL / f_CLK

M_PLL, M_MS

(32, 27.77777777777778)

In [4]:
# MS as fraction
a = int(M_MS)
frac = M_MS - a

# Basic limits.
lohi_b = [0, 2**20]
lohi_c = [1, 2**20]

b, c = clock.rational_approximation(frac, lohi_num=lohi_b, lohi_den=lohi_c)

a, b, c, a+b/c, M_MS

(27, 7, 9, 27.77777777777778, 27.77777777777778)

# Define

In [5]:
C = clock.Clock()

In [21]:
C.reset()

In [22]:
C.config_input()

In [23]:
f_PLL = 800e6
C.config_PLL(f_PLL)

In [24]:
ix_MS = 0
f_MS = 25e6
src_PLL = 'A'
C.config_MS(f_MS, ix_MS, src_PLL=src_PLL)

In [25]:
ix_CLK = ix_MS
C.config_CLK(ix_CLK)

In [26]:
s_PLL = 'A'
C.soft_reset_PLL(s_PLL)

In [27]:
C.enable_output(ix_CLK)

# Output Stage (Clocks 1 - 7)

In [10]:
# # CLK source: match output CLK with corresponding Stage-2 MultiSynth.  More complicated options are available.
# C['CLK0_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK1_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK2_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK3_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK4_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK5_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK6_SRC'] = clock.constants.CLK_SRC_MS
# C['CLK7_SRC'] = clock.constants.CLK_SRC_MS

# C['CLK0_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK1_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK2_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK3_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK4_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK5_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK6_INV'] = clock.constants.CLK_INV_FALSE
# C['CLK7_INV'] = clock.constants.CLK_INV_FALSE

# # Clock drive current (what's the best setting here?)
# C['CLK0_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK1_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK2_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK3_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK4_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK5_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK6_IDRV'] = clock.constants.CLK_IDRV_8
# C['CLK7_IDRV'] = clock.constants.CLK_IDRV_8

# # Initial phase values
# C['CLK0_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK1_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK2_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK3_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK4_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# C['CLK5_PHOFF'] = clock.constants.CLK_PHOFF_ZERO
# # C['CLK6_PHOFF'] = clock.constants.CLK_PHOFF_ZERO  # does not exist
# # C['CLK7_PHOFF'] = clock.constants.CLK_PHOFF_ZERO  # does not exist


# # Clock final divide (unity)
# C['R0_DIV'] = clock.constants.R_DIV_1
# C['R1_DIV'] = clock.constants.R_DIV_1
# C['R2_DIV'] = clock.constants.R_DIV_1
# C['R3_DIV'] = clock.constants.R_DIV_1
# C['R4_DIV'] = clock.constants.R_DIV_1
# C['R5_DIV'] = clock.constants.R_DIV_1
# C['R6_DIV'] = clock.constants.R_DIV_1
# C['R7_DIV'] = clock.constants.R_DIV_1